In [1]:
import dask
import dask.dataframe as ddf
import os
from glob import glob
import re
import pandas as pd
import numpy as np
import pickle

In [2]:
user_group = pd.read_csv('result/cluster-result-original-distance-0.2.csv', sep="\t",encoding='utf-16')

In [3]:
df = pd.read_parquet('../../../intermediate-result/TCM/editors-with-sig-contrib-at-least-10')

# top 10 articles by sig contributions

In [5]:
# read sig contribution info and only keep rvs with selected editors

In [6]:
x = pd.read_parquet('../../../intermediate-result/TCM/sig-contrib-info')

In [7]:
user_contribution = x[x['contributor.username'].isin(df.index)]

In [8]:
user_contribution = user_contribution.set_index(['contributor.username','page.title'])

In [9]:
user_contribution.to_parquet('../../../intermediate-result/TCM/editor-page-sig-contributions')

In [10]:
def top_ranked_articles(group):
    curr_user_group = user_group[user_group['group'] == group]
    all_users = curr_user_group['contributor.username'].unique()
    selected_df = user_contribution[user_contribution.index.get_level_values(0).isin(all_users)]
    res = {}
    for index in selected_df.index:
        if not index[1].lower().startswith('talk:'):
            if index[1] not in res:
                res[index[1]] = selected_df.loc[index]['sig.contributions']
            else:
                res[index[1]] += selected_df.loc[index]['sig.contributions']
    res = dict(sorted(res.items(), key=lambda item: item[1], reverse=True))
    return {k: res[k] for k in list(res)[:10]}

In [11]:
all_groups = user_group['group'].unique()

In [12]:
fin = {}

In [13]:
for group in all_groups:
    fin[group] = top_ranked_articles(group)

In [14]:
x = pd.DataFrame.from_dict(fin, orient="index").stack().to_frame()

In [15]:
x.reset_index(inplace=True)

In [16]:
x = x.rename(columns={'level_0':'group','level_1':'article',0:'num_sig_contributions'})

In [17]:
x = x.sort_values(['group', 'num_sig_contributions'],ascending=False)

In [18]:
x.to_csv('result/TCM-top-10-articles-for-each-group-by-sig-contributions.csv', index=False,sep="\t",encoding='utf-16')

# top 10 articles by revert

In [20]:
group_rv = pd.read_csv('result/TCM-group-revert-graph.csv', sep="\t",encoding='utf-16').set_index('Unnamed: 0')

In [21]:
revert_info = pd.read_parquet('../../../intermediate-result/TCM/TCM-revert-user-index-info')

In [22]:
revert_info = revert_info.merge(user_group,left_on='user1',right_on='contributor.username').rename(columns={'group':'group1'})

In [23]:
revert_info = revert_info.merge(user_group,left_on='user2',right_on='contributor.username').rename(columns={'group':'group2'})

In [24]:
revert_to = revert_info.groupby(['group1','page.title']).agg({'group1':'count'}).rename(columns={'group1':'do_revert_to'})

In [25]:
revert_from = revert_info.groupby(['group2','page.title']).agg({'group1':'count'}).rename(columns={'group1':'receive_revert_from'})

In [26]:
revert_to.index.names = ['group','page.title']
revert_from.index.names = ['group','page.title']

In [27]:
res = pd.concat([revert_to.reset_index(),revert_from.reset_index()]).fillna(0)

In [28]:
res = res.groupby(['group','page.title']).agg({'do_revert_to':'sum','receive_revert_from':'sum'})

In [29]:
res['mutual_revert'] = res['do_revert_to'] + res['receive_revert_from']

In [30]:
fin = res.sort_values(['group','mutual_revert'],ascending=False).groupby(['group']).head(10)

In [31]:
fin.reset_index(inplace=True)

In [32]:
fin.to_csv('result/TCM-top-10-articles-for-each-group-by-mutual-revert.csv', index=False,sep="\t",encoding='utf-16')